In [5]:
import numpy as np
import os
import sys
import multiprocessing
import time

# PyTorch dependencies
import torch as pt
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

# local dependencies
#from bps import bps
from sembps.bps import aptbps
from modelnet40 import load_modelnet40

N_SAMPLES = 1000

MAIN_PATH = os.path.join(os.sep, 'aptbps-code')
MOD40_PATH = os.path.join(MAIN_PATH, 'modelnet40')

LOGS_PATH = os.path.join(MOD40_PATH, 'logs')
DATA_PATH = os.path.join(MOD40_PATH, 'data')

BPS_CACHE_FILE = os.path.join(DATA_PATH, 'bps_mlp_data.npz')
APTBPS_CACHE_FILE = os.path.join(DATA_PATH, 'aptbps_mlp_data.npz')

N_MODELNET_CLASSES = 40

N_BPS_POINTS = 512
BPS_RADIUS = 1

N_CPUS = multiprocessing.cpu_count()
N_GPUS = torch.cuda.device_count()

if N_GPUS > 0:
    DEVICE = 'cuda'
    print("GPU device found...")
else:
    DEVICE = 'cpu'
    print("GPU device not found, using %d CPU(s)..." % N_CPUS)

if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)


class ShapeClassifierMLP(nn.Module):

    def __init__(self, n_features, n_classes, hsize1=512,  hsize2=512, dropout1=0.8, dropout2=0.8):
        super(ShapeClassifierMLP, self).__init__()

        self.bn0 = nn.BatchNorm1d(n_features)
        self.fc1 = nn.Linear(in_features=n_features, out_features=hsize1)
        self.bn1 = nn.BatchNorm1d(hsize1)
        self.do1 = nn.Dropout(dropout1)
        self.fc2 = nn.Linear(in_features=hsize1, out_features=hsize2)
        self.bn2 = nn.BatchNorm1d(hsize2)
        self.do2 = nn.Dropout(dropout2)
        self.fc3 = nn.Linear(in_features=hsize2, out_features=n_classes)

    def forward(self, x):

        x = self.bn0(x)
        x = self.do1(self.bn1(F.relu(self.fc1(x))))
        x = self.do2(self.bn2(F.relu(self.fc2(x))))
        x = self.fc3(x)

        return x


def fit(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()


def test(model, device, test_loader, epoch_id):
    model.eval()
    test_loss = 0
    n_test_samples = len(test_loader.dataset)
    n_correct = 0
    with pt.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            n_correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= n_test_samples
    test_acc = 100.0 * n_correct / n_test_samples
    print(
        "Epoch {} test loss: {:.4f}, test accuracy: {}/{} ({:.2f}%)".format(epoch_id, test_loss, n_correct, n_test_samples, test_acc))

    return test_loss, test_acc

from sklearn.neighbors import NearestNeighbors

def prepare_data_loaders():

    if not os.path.exists(BPS_CACHE_FILE):

        # load modelnet point clouds
        xtr, ytr, xte, yte = load_modelnet40(root_data_dir=DATA_PATH)

        # this will normalise your point clouds and return scaler parameters for inverse operation
        xtr_normalized = aptbps.normalize(xtr)
        xte_normalized = aptbps.normalize(xte)

        # this will encode your normalised point clouds with random basis of 512 points,
        # each BPS cell containing l2-distance to closest point
        start = time.time()
        print("converting data to BPS representation..")
        print("number of basis points: %d" % N_BPS_POINTS)
        print("BPS sampling radius: %f" % BPS_RADIUS)
        print("converting train..")
        xtr_bps = aptbps.encode(xtr_normalized[0:N_SAMPLES], bps_arrangement="random", n_bps_points=N_BPS_POINTS, bps_cell_type='dists', radius=BPS_RADIUS)
        
        xtr_aptbps = aptbps.adaptive_encode(xtr_normalized[0:N_SAMPLES], bps_arrangement="random", n_parts=2, n_bps_points=N_BPS_POINTS, bps_cell_type='dists', radius=BPS_RADIUS)
        print("converting test..")
        xte_bps = aptbps.encode(xte_normalized[0:N_SAMPLES], bps_arrangement="random", n_bps_points=N_BPS_POINTS, bps_cell_type='dists', radius=BPS_RADIUS) # test
        
        xte_aptbps = aptbps.adaptive_encode(xtr_normalized[0:N_SAMPLES], bps_arrangement="random", n_parts=2, n_bps_points=N_BPS_POINTS, bps_cell_type='dists', radius=BPS_RADIUS)
        end = time.time()
        total_training_time = (end - start) / 60
        print("conversion finished. ")
        print("saving cache file for future runs..")

        np.savez(APTBPS_CACHE_FILE, xtr=xtr_aptbps,
                 ytr=ytr[0:N_SAMPLES], xte=xte_aptbps, yte=yte[0:N_SAMPLES])

        np.savez(BPS_CACHE_FILE, xtr=xtr_bps, ytr=ytr[0:N_SAMPLES],
                 xte=xte_bps, yte=yte[0:N_SAMPLES])

    else:
        print("loading converted data from cache..")
        data = np.load(BPS_CACHE_FILE)
        xtr_bps = data['xtr']
        ytr = data['ytr']
        xte_bps = data['xte']
        yte = data['yte']


# For kde
from scipy import stats

def main():
    prepare_data_loaders()


from functools import partial
import multiprocessing
import numpy as np
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

# For kde
from scipy import stats

if __name__ == '__main__':
    main()


GPU device found...
loading ModelNet40 point clouds...
downloading ModelNet40 data..


100.0% 435216384 / 435212151


unzipping files..
File Name                                             Modified             Size
modelnet40_ply_hdf5_2048/                      2017-02-02 13:28:26            0
modelnet40_ply_hdf5_2048/ply_data_train_2_id2file.json 2017-02-01 14:36:12        52667
modelnet40_ply_hdf5_2048/ply_data_train2.h5    2017-02-01 14:29:06     72245751
modelnet40_ply_hdf5_2048/ply_data_train4.h5    2017-02-01 14:29:06     58634835
modelnet40_ply_hdf5_2048/ply_data_train1.h5    2017-02-01 14:29:06     73196621
modelnet40_ply_hdf5_2048/train_files.txt       2017-02-01 14:37:42          245
modelnet40_ply_hdf5_2048/ply_data_train_4_id2file.json 2017-02-01 14:36:24        42383
modelnet40_ply_hdf5_2048/ply_data_test1.h5     2017-02-01 14:29:06     14356248
modelnet40_ply_hdf5_2048/ply_data_test0.h5     2017-02-01 14:29:06     70992141
modelnet40_ply_hdf5_2048/ply_data_test_1_id2file.json 2017-02-01 14:35:50        11135
modelnet40_ply_hdf5_2048/ply_data_train_1_id2file.json 2017-02-01 14:36:04     

100%|██████████| 83/83 [00:00<00:00, 334.16it/s]

100%|██████████| 83/83 [00:00<00:00, 347.10it/s]


using 12 available CPUs for BPS encoding..


...

converting test..
using 12 available CPUs for BPS encoding..


100%|██████████| 84/84 [00:00<00:00, 334.67it/s]

100%|██████████| 83/83 [00:00<00:00, 321.52it/s]

100%|██████████| 83/83 [00:00<00:00, 336.42it/s]


using 12 available CPUs for BPS encoding..


100%|██████████| 83/83 [00:25<00:00,  3.23it/s]

100%|██████████| 84/84 [00:26<00:00,  3.23it/s]


conversion finished. 
saving cache file for future runs..
